In [1]:
%pylab inline
sc.version

Populating the interactive namespace from numpy and matplotlib


u'1.3.1'

In [90]:
from xmltodict import parse as parse_xml
from pyspark.sql import SQLContext, Row
from dateutil.parser import parse as parse_date
import re
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from collections import Counter

In [109]:
def xml_to_rdd(file_name):
    def infer_type(input):
        int_re = re.compile('^[\-]?[0-9]+$')
        date_re = re.compile('^[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{2}.[0-9]{3}$')
        int_type = (int_re.match, int)
        date_type = (date_re.match, parse_date)
        default_type = (lambda x: True, lambda x: x)

        for f, c in [int_type, date_type, default_type]:
            if f(input):
                return c(input)

    def clean_xml_dict(xd):
        return {
                key[1:]: infer_type(value.strip())
                for key, value in xd.items() if key.startswith('@')
            }

    return sc.textFile(file_name)\
    .filter(lambda line: line.strip().startswith('<row'))\
    .map(lambda line: parse_xml(line))\
    .map(lambda xml_dict: xml_dict['row'])\
    .map(clean_xml_dict)

def extract_fields(row, wanted):
    return {
        k: row.get(k, None)
        for k in wanted
    }

def rdd_to_df(rdd, columns, sql_context, schema=None):
    rows = rdd.map(lambda r: extract_fields(r, columns))\
    .map(lambda r: Row(**r))
    
    return sql_context.createDataFrame(rows, schema, 0.1)

def add_to_dict(input, key, value):
    input[key] = value
    return input

In [35]:
sql_context = SQLContext(sc)
post_age = udf(lambda dt: (datetime(2014,9,14) - dt).days, IntegerType())

In [5]:
posts = xml_to_rdd('/Users/friso/Downloads/se/Posts.xml')
posts.take(1)

[{u'AcceptedAnswerId': 393,
  u'AnswerCount': 4,
  u'Body': u"<p>My fianc\xe9e and I are looking for a good Caribbean cruise in October and were wondering which islands are best to see and which Cruise line to take?</p>\n\n<p>It seems like a lot of the cruises don't run in this month due to Hurricane season so I'm looking for other good options.</p>\n\n<p><strong>EDIT</strong> We'll be travelling in 2012.</p>",
  u'ClosedDate': datetime.datetime(2013, 2, 25, 23, 52, 47, 953000),
  u'CommentCount': 4,
  u'CreationDate': datetime.datetime(2011, 6, 21, 20, 19, 34, 730000),
  u'FavoriteCount': 1,
  u'Id': 1,
  u'LastActivityDate': datetime.datetime(2012, 5, 24, 14, 52, 14, 760000),
  u'LastEditDate': datetime.datetime(2011, 12, 28, 21, 36, 43, 910000),
  u'LastEditorUserId': 101,
  u'OwnerUserId': 9,
  u'PostTypeId': 1,
  u'Score': 8,
  u'Tags': u'<caribbean><cruising><vacation>',
  u'Title': u'What are some Caribbean cruises for October?',
  u'ViewCount': 309}]

In [68]:
posts_df = rdd_to_df(posts,
                     ['Id', 'Score', 'CreationDate'],
                     sql_context)

In [69]:
votes = xml_to_rdd('/Users/friso/Downloads/se/Votes.xml')
votes.take(1)

[{u'CreationDate': datetime.datetime(2011, 6, 21, 0, 0),
  u'Id': 1,
  u'PostId': 1,
  u'VoteTypeId': 2}]

In [87]:
votes_df = rdd_to_df(votes,
                     ['VoteTypeId', 'CreationDate', 'PostId'],
                     sql_context).withColumnRenamed('CreationDate', 'VoteDate')

In [88]:
posts_votes_df = posts_df.join(votes_df, posts_df.Id == votes_df.PostId, 'inner')

In [82]:
posts_votes_df.show()

CreationDate         Id  Score CreationDate         PostId VoteTypeId
2011-06-21 20:48:... 31  8     2011-06-21 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2011-06-21 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2011-06-21 00:00:... 31     16        
2011-06-21 20:48:... 31  8     2011-06-22 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2011-06-27 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2011-10-31 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2011-11-25 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2011-12-01 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2013-03-16 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2013-12-06 00:00:... 31     2         
2011-06-21 20:48:... 31  8     2013-12-06 00:00:... 31     3         
2011-06-22 17:43:... 231 4     2011-06-22 00:00:... 231    2         
2011-06-22 17:43:... 231 4     2011-06-23 00:00:... 231    2         
2011-06-22 17:43:...

In [89]:
posts_votes_df.rdd.take(5)

[Row(CreationDate=datetime.datetime(2011, 6, 21, 20, 48, 29, 23000), Id=31, Score=8, VoteDate=datetime.datetime(2011, 6, 21, 0, 0), PostId=31, VoteTypeId=2),
 Row(CreationDate=datetime.datetime(2011, 6, 21, 20, 48, 29, 23000), Id=31, Score=8, VoteDate=datetime.datetime(2011, 6, 21, 0, 0), PostId=31, VoteTypeId=2),
 Row(CreationDate=datetime.datetime(2011, 6, 21, 20, 48, 29, 23000), Id=31, Score=8, VoteDate=datetime.datetime(2011, 6, 21, 0, 0), PostId=31, VoteTypeId=16),
 Row(CreationDate=datetime.datetime(2011, 6, 21, 20, 48, 29, 23000), Id=31, Score=8, VoteDate=datetime.datetime(2011, 6, 22, 0, 0), PostId=31, VoteTypeId=2),
 Row(CreationDate=datetime.datetime(2011, 6, 21, 20, 48, 29, 23000), Id=31, Score=8, VoteDate=datetime.datetime(2011, 6, 27, 0, 0), PostId=31, VoteTypeId=2)]

In [97]:
def prepare_post_ft( (post_id, rows) ):
    cnt = Counter()
    cnt.update([
            row.VoteTypeId
            for row in rows if (row.VoteDate - row.CreationDate).days <= 7
        ])
    
    return add_to_dict(dict(cnt), 'PostId', post_id)
    

ft = posts_votes_df.rdd.groupBy(lambda r: r.PostId).map(prepare_post_ft)

In [110]:
ft_fields = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', 'PostId']
ft_df = rdd_to_df(ft,
                 ft_fields,
                 sql_context,
                 [StructField(f, LongType(), True) for f in ft_fields]).na.fill(0)
ft_df.show()

TypeError: StructField(1,LongType,true) is not JSON serializable

In [100]:
?sql_context.createDataFrame

In [79]:
votes_ft = sql_context.sql("""
select PostId, VoteTypeId, count(*) as Cnt, max(CreationDate) as MaxVoteDate
from votes group by PostId, VoteTypeId""")

In [80]:
posts_votes_ft = posts_df.join(votes_ft, posts_df.Id == votes_ft.PostId, 'inner')

In [78]:
posts_votes_ft.show()

CreationDate         Id   Score PostId VoteTypeId Cnt MaxDate             
2011-06-21 20:48:... 31   8     31     2          9   2013-12-06 00:00:...
2011-06-21 20:48:... 31   8     31     3          1   2013-12-06 00:00:...
2011-06-21 20:48:... 31   8     31     16         1   2011-06-21 00:00:...
2011-06-22 17:43:... 231  4     231    2          4   2013-01-18 00:00:...
2011-06-24 20:10:... 431  4     431    2          5   2011-10-30 00:00:...
2011-06-24 20:10:... 431  4     431    3          1   2011-06-26 00:00:...
2011-06-29 13:35:... 631  3     631    1          1   2011-06-29 00:00:...
2011-06-29 13:35:... 631  3     631    2          3   2011-07-15 00:00:...
2011-07-06 10:45:... 831  7     831    2          7   2014-08-11 00:00:...
2011-07-16 11:49:... 1031 11    1031   2          11  2014-05-20 00:00:...
2011-07-16 11:49:... 1031 11    1031   5          2   2012-01-19 00:00:...
2011-07-24 16:24:... 1231 5     1231   1          1   2011-08-15 00:00:...
2011-07-24 16:24:... 1231